In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import concurrent.futures
import math

from tqdm import tqdm
import numpy as np
import polars as pl
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine

In [4]:
TOP_N = 25

# MRR@100

In [5]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/train_task1.parquet")

In [6]:
candidates = pl.concat([
    pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/similar_products_19_DE.parquet").sort(["locale", "item", "bert_distance"], descending=[False, False, False]).groupby(["locale", "item"], maintain_order=True).head(TOP_N),
    pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/similar_products_19_UK.parquet").sort(["locale", "item", "bert_distance"], descending=[False, False, False]).groupby(["locale", "item"], maintain_order=True).head(TOP_N),
    pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/similar_products_19_JP.parquet").sort(["locale", "item", "bert_distance"], descending=[False, False, False]).groupby(["locale", "item"], maintain_order=True).head(TOP_N),
])

In [7]:
candidates.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/similar_products_19.parquet")

In [8]:
# last_itemの抽出
last_item_list = []
prev_items_list = train["prev_items"].to_list()
for prev_items in prev_items_list:
    last_item_list.append(prev_items[-1])
train = train.with_columns(pl.Series(name="last_item", values=last_item_list))

In [9]:
train = train[["locale", "session_id", "last_item", "next_item"]]

In [10]:
label_lists = []
n_rows = 500_000
for df in tqdm(train.iter_slices(n_rows=n_rows), total=math.ceil(train.height/n_rows)): # specify "total" parameter to display tqdm progress bar 
    # process data
    df = df.join(candidates, left_on=["last_item", "locale"], right_on=["item", "locale"], how="left")
    df = df.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
    label_lists.extend(df.groupby("session_id", maintain_order=True).all()["label"].to_list())

100%|██████████| 7/7 [00:34<00:00,  4.93s/it]


In [11]:
# MRRの計算
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.16865
